In [40]:
from io import StringIO

#from lexicon_extraction import *


def test_count_sentence_long():
    # We can use a StringIO object to simulate reading from a file
    # without actually creating a temporary file for the tests
    input = StringIO("""a b c
d c a
""")
    assert count_sentences_with_word_long(input) == {'a': 2, 'c': 2, 'b': 1, 'd': 1}


def test_count_sentence():

    input = StringIO("""a b c
d c a
""")
    assert count_sentences_with_word(input) == {'a': 2, 'c': 2, 'b': 1, 'd': 1}


def test_build_cooc_long():

    src = StringIO(""" a b c a
d c a
""")
    tgt = StringIO("""A B C A
D C A
""")

    res = build_cooc_table_long(src, tgt)
    expected_res = {('c', 'C'): 2,
                    ('c', 'A'): 2,
                    ('a', 'C'): 2,
                    ('a', 'A'): 2,
                    ('c', 'B'): 1,
                    ('b', 'B'): 1,
                    ('b', 'C'): 1,
                    ('b', 'A'): 1,
                    ('a', 'B'): 1,
                    ('c', 'D'): 1,
                    ('a', 'D'): 1,
                    ('d', 'A'): 1,
                    ('d', 'D'): 1,
                    ('d', 'C'): 1}

    assert res == expected_res


def test_build_cooc():

    src = StringIO(""" a b c a
d c a
""")
    tgt = StringIO("""A B C A
D C A
""")

    res = build_cooc_table(src, tgt)
    expected_res = {('c', 'C'): 2,
                    ('c', 'A'): 2,
                    ('a', 'C'): 2,
                    ('a', 'A'): 2,
                    ('c', 'B'): 1,
                    ('b', 'B'): 1,
                    ('b', 'C'): 1,
                    ('b', 'A'): 1,
                    ('a', 'B'): 1,
                    ('c', 'D'): 1,
                    ('a', 'D'): 1,
                    ('d', 'A'): 1,
                    ('d', 'D'): 1,
                    ('d', 'C'): 1}
    
    assert res == expected_res



In [56]:

def validLine(l):
    return l.strip()
def count_sentences_with_word(f) :
    index_line = 0
    res = {}
    for line in f:
        if validLine(line) :
            index_line += 1
            for word in line.split() :
                if word in res :
                    res.get(word).add(index_line)
                else :
                    res[word]={index_line}
    
    return res

In [68]:
english = 'english.corpus'
french = 'french.corpus'
def build_cooc_table(f1, f2):
    r1 = count_sentences_with_word(f1)
    r2 = count_sentences_with_word(f2)
    res = {}
    for k1,v1 in r1.items():
        for k2,v2 in r2.items():
            comm = len(v1&v2)
            if(comm != 0):
                res[(k1,k2)] = comm
    return res

def build_cooc_table_by_sentences(r1, r2):
    res = {}
    for k1,v1 in r1.items():
        for k2,v2 in r2.items():
            comm = len(v1&v2)
            if(comm != 0):
                res[(k1,k2)] = comm
    return res
# sort dict by the value 
def dump(d):
    return dict(sorted(d.items(), key=lambda e:e[1], reverse=True))

In [43]:
s = {1,2,3}
ss = {2,3}
s & ss

{2, 3}

In [44]:
test_build_cooc()

In [45]:
# n（-a, -b) = N - n(a) - n(b) + n(a, b)
# n (-a,  b) = n(b) - n(a ,b)
# n (a,  -b) = n(a) - n(a ,b)

In [99]:
import math
def likelihood(n, na, nb, nab):
    nnanb = n - na - nb + nab # nombre de non a et non b
    nnab  = nb - nab          # nombre de non a et b
    nanb  = na - nab          # nombre de a et non b
    
    pa = na/n                 # probabilite de a
    pb = nb/n                 # probabilite de b
    pab = nab/n               # probabilite de a et b
    
    pna = 1-pa
    pnb = 1-pb
    pnanb = nnanb/n
    pnab = nnab/n
    panb = nanb/n
    
    #print("pa : ", pa, "  pb : ", pb, "  pab : ", pab, "  pna : ", pna, "  pnb : ", pnb, "  pnanb : ", pnanb, "  pnab : ", pnab, "  panb : ", panb)
    
    llab = pab*math.log(pab/(pa*pb))
    llanb = panb*math.log(panb/(pa*pnb))
    llnab = pnab*math.log(pnab/(pna*pb))
    llnanb = pnanb*math.log(pnanb/(pna*pnb))
    
    gcare = 2*n*(llab+llanb+llnab+llnanb)
    
    return gcare


In [106]:
def len_file(filename):
    return len([1 for line in open(filename, 'r', encoding='UTF-8') if validLine(line)])

def computer_all(f1, f2, n):
    r1 = count_sentences_with_word(f1)
    r2 = count_sentences_with_word(f2)
    dic = dump(build_cooc_table_by_sentences(r1, r2))
    res = {}
    for k,v in dic.items():
        na = len(r1.get(k[0]))
        nb = len(r2.get(k[1]))
        nab = v
        try:
            like = likelihood(n, na, nb, nab)
        except ValueError:
            continue
        else:
            res[k] = like
    return dump(res)
n = len_file(french)
f1 = open(french, 'r', encoding='UTF-8')
f2 = open(english, 'r', encoding='UTF-8')
res = computer_all(f1,f2,n)
print('fin')
print(list((res.items()))[:100])

fin
[(('et', 'and'), 1455.6536120221535), (('Barbicane', 'Barbicane'), 1144.136020751402), ((',', ','), 996.8393553059196), (('Lune', 'moon'), 753.9648776623804), (('?', '?'), 747.4566206460229), (('.', '.'), 737.2016866451971), (('Ardan', 'Ardan'), 698.8103804145431), (('projectile', 'projectile'), 607.5822080489218), (('de', 'of'), 603.9962630684352), (('Michel', 'Ardan'), 499.99106288902516), (('un', 'a'), 480.3433627179802), (('!', '!'), 475.5435673702657), (('Michel', 'Michel'), 449.1437745878284), (('est', 'is'), 430.6667821501907), (('pas', 'not'), 422.5511828120804), (('Nicholl', 'Nicholl'), 416.8528695906571), (('J.-T', 'T'), 413.8929720426891), (('J.-T', 'Maston'), 409.03129766803926), (('par', 'by'), 395.6295016354359), (('J.-T', 'J'), 392.7794323376008), (('je', 'I'), 392.0054711444843), (('la', 'the'), 391.19296712998323), (('Ardan', 'Michel'), 383.8519769227315), (('pieds', 'feet'), 383.5627356255816), (('vous', 'you'), 383.5238996819363), (('Maston', 'T'), 378.3365218156

In [107]:
di = {
    'b':2,
    'a':1,
    'c':4,
    'd':3
}
# sort dict by the key, res : list de tuple
print([(k,di[k]) for k in sorted(di.keys())])

# sort dict by the key, res : dict
print({k:di[k] for k in sorted(di.keys())})

# sort dict by the value
print (dict(sorted(di.items(), key=lambda d: d[1], reverse=True)))

print(sorted(di.items()))

[('a', 1), ('b', 2), ('c', 4), ('d', 3)]
{'c': 4, 'd': 3, 'b': 2, 'a': 1}
{'a': 1, 'b': 2, 'c': 4, 'd': 3}
[('a', 1), ('b', 2), ('c', 4), ('d', 3)]


In [ ]:
# conclusion : le plus grand de likelihood, le sens entre deux mots anglais et francais est le plus proche